### Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп».   
Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию.   
Обоснуйте корректность его работы.  
Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.   
Подбирать наилучшую модель не требуется.  

  ### Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from numpy import linalg
from sklearn.model_selection import train_test_split

In [2]:
try:
    data=pd.read_csv(r"C:\Users\Admin\Desktop\insurance.csv")
except:
    data=pd.read_csv('/datasets/insurance.csv')

In [3]:
print(data.shape)

(5000, 5)


In [4]:
print(data.head())

   Пол  Возраст  Зарплата  Члены семьи  Страховые выплаты
0    1     41.0   49600.0            1                  0
1    0     46.0   38000.0            1                  1
2    0     29.0   21000.0            0                  0
3    0     21.0   41700.0            2                  0
4    1     28.0   26100.0            0                  0


In [5]:
data['Пол'].unique()  #категориальный признак Пол уже закодирован

array([1, 0])

In [6]:
data['Члены семьи'].unique() #количество членов семьи

array([1, 0, 2, 4, 3, 5, 6])

In [7]:
data['Страховые выплаты'].unique() #количество страховых выплат

array([0, 1, 2, 3, 5, 4])

In [8]:
data.info() #пропусков в данных нет

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [9]:
print(data[data.duplicated()])

      Пол  Возраст  Зарплата  Члены семьи  Страховые выплаты
281     1     39.0   48100.0            1                  0
488     1     24.0   32900.0            1                  0
513     0     31.0   37400.0            2                  0
718     1     22.0   32600.0            1                  0
785     0     20.0   35800.0            0                  0
...   ...      ...       ...          ...                ...
4793    1     24.0   37800.0            0                  0
4902    1     35.0   38700.0            1                  0
4935    1     19.0   32700.0            0                  0
4945    1     21.0   45800.0            0                  0
4965    0     22.0   40100.0            1                  0

[153 rows x 5 columns]


Дубликаты оставим, так как у нас нет уникального идентификатора человека, а признаков не так много, значит одинаковые записи могут соответствовать разным людям. 

In [10]:
data.columns=['gender','age','salary','family_members','insurance_payments'] #переименовала колонки
print(data.head())

   gender   age   salary  family_members  insurance_payments
0       1  41.0  49600.0               1                   0
1       0  46.0  38000.0               1                   1
2       0  29.0  21000.0               0                   0
3       0  21.0  41700.0               2                   0
4       1  28.0  26100.0               0                   0


In [11]:
data['salary']=data['salary']/1000 #зарплата в другом масштабе по сравнению с другими признаками

### Умножение матриц

In [12]:
features = data.drop('insurance_payments', axis=1)
target = data['insurance_payments']

In [13]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)


In [14]:

class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]
        return self.w,self.w0
    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0
model = LinearRegression()
f=model.fit(features_train, target_train)
predictions = model.predict(features_test)
print("Исходный датасет")
print('r2_score=',r2_score(target_test, predictions))

w_original=f[0]
w0_original=f[1]


Исходный датасет
r2_score= 0.43522757127026657


Размерность исходной матрицы признаков m=5000,n=4, значит нужна матрица размером 4 на 4. 

In [15]:
new = np.random.randint(10,size = (4,4)) #матрицу заполняем рандомными числами

In [16]:
print(new)

[[9 8 0 3]
 [2 2 5 0]
 [5 0 4 5]
 [8 3 6 3]]


In [17]:
linalg.det(new)

-525.9999999999997

In [18]:
np.linalg.inv(new) #матрица обратимая

array([[-0.07414449, -0.34220532, -0.21102662,  0.42585551],
       [ 0.16920152,  0.31939163,  0.09695817, -0.33079848],
       [-0.03802281,  0.20912548,  0.04562738, -0.03802281],
       [ 0.10456274,  0.17490494,  0.37452471, -0.39543726]])

In [19]:
np.linalg.matrix_rank(new) #ранг матрицы не меньше 4

4

Случайно сгенерованнная матрица оказалась невыроженной и обратимой (вероятность, что она окажется вырожденной или необратимой была очень низкой) 

In [20]:
data_new=features.values @ new 

In [21]:
print(data_new)

[[347.   93.  409.4 254. ]
 [290.   95.  388.  193. ]
 [163.   58.  229.  105. ]
 ...
 [225.5  46.  247.6 175.5]
 [240.5  61.  258.8 175.5]
 [276.   67.  308.4 209. ]]


In [22]:
data_new_df=pd.DataFrame(data_new, columns=['gender','age','salary','family_members'])

In [23]:
data_new_df['insurance_payments']=data['insurance_payments']

In [24]:
print(data_new_df.head())
print(data.head())

   gender   age  salary  family_members  insurance_payments
0   347.0  93.0   409.4           254.0                   0
1   290.0  95.0   388.0           193.0                   1
2   163.0  58.0   229.0           105.0                   0
3   266.5  48.0   283.8           214.5                   0
4   195.5  64.0   244.4           133.5                   0
   gender   age  salary  family_members  insurance_payments
0       1  41.0    49.6               1                   0
1       0  46.0    38.0               1                   1
2       0  29.0    21.0               0                   0
3       0  21.0    41.7               2                   0
4       1  28.0    26.1               0                   0


In [25]:
features = data_new_df.drop('insurance_payments', axis=1)
target = data_new_df['insurance_payments']

In [26]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)

In [27]:

class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]
        return self.w,self.w0
    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0
    
model = LinearRegression()
f=model.fit(features_train, target_train)
predictions = model.predict(features_test)
print("При умножении на не вырожденную матрицу")
print('r2_score=',r2_score(target_test, predictions))

w_modified=f[0]
w0_modified=f[1]

При умножении на не вырожденную матрицу
r2_score= 0.4352275712702621


Теперь возьмем вырожденную матрицу, добавив высокую степень корреляции между элементами матрицы.

In [28]:
new = np.array([
    [0.2, 1.2, 2,4],
    [0.5, 0.8, 3,6],
    [0.8, 1.6, 4,8],
[0.8, 1.6, 5,10]])


In [29]:
print(new)

[[ 0.2  1.2  2.   4. ]
 [ 0.5  0.8  3.   6. ]
 [ 0.8  1.6  4.   8. ]
 [ 0.8  1.6  5.  10. ]]


In [30]:
linalg.det(new)

0.0

In [31]:
features = data.drop('insurance_payments', axis=1)
target = data['insurance_payments']

In [32]:
data_new=features.values @ new 

In [33]:
data_new_df=pd.DataFrame(data_new, columns=['gender','age','salary','family_members'])

In [34]:
data_new_df['insurance_payments']=data['insurance_payments']

In [35]:
features = data_new_df.drop('insurance_payments', axis=1)
target = data_new_df['insurance_payments']

In [36]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)

In [37]:

class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.pinv(X.T.dot(X)).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]
        return self.w,self.w0
    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0
    
model = LinearRegression()
f=model.fit(features_train, target_train)
predictions = model.predict(features_test)
print("При умножении на вырожденную матрицу")
print('r2_score=',r2_score(target_test, predictions))


При умножении на вырожденную матрицу
r2_score= 0.4239561081586166


Цель - максимизация R2, при умножении на вырожденному матрицу, качество модели снижается. 

### Алгоритм преобразования

В качестве ключа будем использовать матрицу new. 
Посмотрим как связаны матрица признаков исходной и преобразованной матрицы.  

Мы обучили модель линейной регрессии на матрице признаков $features.values$.   
Мы получили веса $w__original$ и $w0__original$.  
Получается целевой признак $target.values$ получается по следующей формуле:  

$$ features.values @ w__original + w0__original  $$ 

Потом матрицу признаков $features.values$ мы умножили на матрицу-ключ $new$.    
Когда мы преобразовали матрицу признаков, умножив ее на $new$,  то получили новые веса $w__modified$ и $w0__modified$.    


Получается, целевой признак $target.values$ получается по следующей формуле:  


$$features.values__modified @ w_modified + w0_modified$$  

$$ features.values__modified = features.values @ new $$
Получается:  

$$features.values @ w__original + w0__original = features.values @ new @ w__modified + w0__modified$$

Видим,что коэффициенты w0_modified и w0_original почти равны, поэтому их можно удалить из формулы доказательства.


$$ features.values @ w__original = features.values @ new @ w__modified  $$
если сократить слева и справа уравнения $features.values$, то получается следующая формула:      
$$w__original= new @ w__modified $$ 
Так как матрица new квадратная, то получается:    
$$w__modified= w__original @ (new)^{-1}  $$

In [38]:
print(w0_modified-w0_original) #разница очень маленькая

3.107514245925813e-13


### Проверка алгоритма

Попробуем расшифровать данные, получив из преобразованной матрицы исходную.

In [39]:
new = np.random.randint(10,size = (4,4))

In [40]:
features = data.drop('insurance_payments', axis=1)

In [41]:
data_new=features.values@new

In [42]:
features_1=data_new @ np.linalg.inv(new) 

In [43]:
print(features_1.round(3)) #небольшая поправка на погрешность модели
print(features.values)
print(features_1.round(3)-features.values)  
# все элементы матрицы признаков совпали, то есть из преобразованной матрицы можно получить исходную, зная ключ


[[ 1.  41.  49.6  1. ]
 [-0.  46.  38.   1. ]
 [ 0.  29.  21.   0. ]
 ...
 [-0.  20.  33.9  2. ]
 [ 1.  22.  32.7  3. ]
 [ 1.  28.  40.6  1. ]]
[[ 1.  41.  49.6  1. ]
 [ 0.  46.  38.   1. ]
 [ 0.  29.  21.   0. ]
 ...
 [ 0.  20.  33.9  2. ]
 [ 1.  22.  32.7  3. ]
 [ 1.  28.  40.6  1. ]]
[[ 0.  0.  0.  0.]
 [-0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 ...
 [-0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]


###  Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [X]  Весь код выполняется без ошибок
- [X]  Ячейки с кодом расположены в порядке исполнения
- [X]  Выполнен шаг 1: данные загружены
- [X]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [X]  Указан правильный вариант ответа
    - [X]  Вариант обоснован
- [X]  Выполнен шаг 3: предложен алгоритм преобразования
    - [X]  Алгоритм описан
    - [X]  Алгоритм обоснован
- [X]  Выполнен шаг 4: алгоритм провере
    - [X]  Алгоритм реализован
    - [X]  Проведено сравнение качества моделей до и после преобразования

Общий вывод:  
    Мы проанализировали данные и нашли способ преобразования исходных данных, который позволяет защитить данные.  
    Данный алгоритм преобразования не ухуджает качество модели предсказания. Качество мы проверями метрикой R2.  
    Метрика R2=0.435   
    В качестве модели была рассмотрена модель линейной регрессии.  
    В качестве алгоритма преобразования выбрано умножение на квадратную невырожденную матрицу.  
    Также выполнено обратное преобразование с целью проверить возможность восстановить исходные данные, зная ключ.  
    Восстановить исходные данные удалось.  
     